## Examining the Relationship Between Economic Instability of Several Countries and BitCoin Market Capitalization

### Authors: Achal Srinivasan and Corrin Fosmire

### Section 1: Setting up Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
import datetime as dt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10,7)

In [2]:
#using IMF API, acquire information for a particular country
def generate_data(country_abbr="US", metric="PXP_IX", resolution="M", start=2009, end=2017):
    data = requests.get("http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/"+resolution+"."+country_abbr+"."+metric+".?startPeriod="+str(start)+"&endPeriod="+str(end)).json()
    df = pd.DataFrame(data['CompactData']['DataSet']['Series']['Obs'])
    df = df.rename(index=str, columns={"@OBS_VALUE" : country_abbr+"."+metric, "@TIME_PERIOD" : "date"})
    return df

In [3]:
#loading economic health data for certain countries"

countries = ["US", "TR", "UY", "VE"]
#United States, Turkey, Uruguay, and Venezuela

metrics = ["PCPI_IX", "ENDE_XDC_USD_RATE", "LE_IX"]
#consumer price index (should indicate inflation), exchange rate (in domestic currency), employment index

print(generate_data(country_abbr="VE",metric="LE_IX").head())

           VE.LE_IX     date
0  95.2845153189527  2009-01
1  96.6061907446666  2009-02
2  98.9964787569285  2009-03
3  100.581520598223  2009-04
4  99.1473978082267  2009-05


In [4]:
def make_dataset(countries, metrics):
    #make first table, so then we can do joins on the rest
    empty = True
    full_data = None
    
    for country in countries:
        for metric in metrics:
            if empty:
                full_data = generate_data(country_abbr=country, metric=metric)
                empty = False
            else:
                metric_data = generate_data(country_abbr=country, metric=metric)
                full_data = full_data.merge(metric_data, on="date")
        time.sleep(5) #avoid throttling
                

    return full_data

In [5]:
financial_data = make_dataset(countries, metrics)

In [6]:
#so we have contiguous time data with no gaps
financial_data = financial_data[financial_data.index > 24]

#deleting status variables we don't really care about
financial_data = financial_data[list(filter(lambda a: not a[0] == "@", list(financial_data.columns.values)))]

In [7]:
financial_data["date"] = financial_data["date"]+"-01"

In [8]:
#loading bitcoin data
btc_data = pd.read_csv("btc.csv")
btc_data = btc_data[btc_data["marketcap(USD)"] != np.nan]

#selecting columns of interest
mc_date = btc_data[["date","marketcap(USD)"]]

#converting to date time format
mc_date['date'] = pd.to_datetime(mc_date['date'])
mc_date = mc_date.dropna()

years = [2013,2014,2015,2016]

mc_date = mc_date[mc_date['date'].dt.year.isin(years) & mc_date['date'].dt.day.isin([1])]
mc_date['date'] = mc_date['date'].dt.strftime(date_format="%Y-%m-%d")
mc_date

/home/infrared/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,date,marketcap(USD)
1573,2013-05-01,1.543334e+09
1604,2013-06-01,1.446526e+09
1634,2013-07-01,1.107511e+09
1665,2013-08-01,1.220624e+09
1696,2013-09-01,1.572969e+09
1726,2013-10-01,1.562891e+09
1757,2013-11-01,2.433531e+09
1787,2013-12-01,1.361969e+10
1818,2014-01-01,9.203028e+09
1849,2014-02-01,1.022621e+10


In [9]:
combined = mc_date.merge(financial_data, on="date")
combined

,date,marketcap(USD),US.PCPI_IX,US.ENDE_XDC_USD_RATE,US.LE_IX,TR.PCPI_IX,TR.ENDE_XDC_USD_RATE,TR.LE_IX,UY.PCPI_IX,UY.ENDE_XDC_USD_RATE,UY.LE_IX,VE.PCPI_IX,VE.ENDE_XDC_USD_RATE,VE.LE_IX
0,2013-05-01,1.543334e+09,106.82830747218,1,103.860155432124,123.357623318386,1.8678,115.555129702857,125.211334849456,20.268,109.694540116312,201.233371509118,6.2842,104.516678174481
1,2013-06-01,1.446526e+09,107.084664225392,1,104.154264795505,124.29932735426,1.9289,116.390947518159,125.75600004537,20.547,109.423522104907,210.695092943353,6.2842,107.427463520202
2,2013-07-01,1.107511e+09,107.126855318944,1,104.34985830856,124.686098654709,1.92585,115.418037891882,126.722010015483,21.5,107.323132516515,217.408157871553,6.2842,107.460299917853
3,2013-08-01,1.220624e+09,107.255721593814,1,103.91552565457,124.557174887892,2.05755,114.803335900734,128.037427847125,22.589,109.423522104907,223.962646462867,6.2842,108.142678440948
4,2013-09-01,1.572969e+09,107.380460479098,1,104.017636973886,125.510089686099,2.036,114.131143795306,129.784467154775,22.038,109.220258596353,233.794379349837,6.2842,108.270747894389
5,2013-10-01,1.562891e+09,107.103925376796,1,103.653056418302,127.769058295964,1.99055,113.423573158013,130.853244142984,21.523,110.168821636271,245.740463395296,6.2842,108.270747894389
6,2013-11-01,2.433531e+09,106.885173728707,1,104.106804604837,127.780269058296,2.01925,112.516998278982,131.120438390037,21.159,110.10106713342,257.58082988283,6.2842,108.596035803177
7,2013-12-01,1.361969e+10,106.876001751848,1,103.853683587943,128.368834080717,2.1362,111.716558995545,130.174981823544,21.389,110.914121167636,263.289578010748,6.2842,109.75382100858
8,2014-01-01,9.203028e+09,107.27360694869,1,103.20865645114,130.908071748879,2.2816,108.152171910183,133.350482682743,22.178,112.065947716109,272.064135318474,6.2842,104.458082835794
9,2014-02-01,1.022621e+10,107.670294947846,1,103.645865480322,131.468609865471,2.2362,110.553489760495,135.55997357183,22.455,113.217774264581,278.460047572901,6.2842,108.420291468253


### Section 2: Exploratory Data Analysis

### Section 3: Modeling

### Section 4: Data Visualization